In [18]:
import io
import random
import string # to process standard python strings
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.stem import WordNetLemmatizer
#nltk.download('punkt') # first-time use only
#nltk.download('wordnet') # first-time use only
#nltk.download('popular', quiet=True)
import openpyxl
from bs4 import BeautifulSoup
import urllib.request
import re
import os


In [19]:
from googlesearch import search

In [20]:
#from google.colab import drive
#drive.mount('/content/drive')

In [21]:
# Keyword Matching
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey","yo ")
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

def greeting(sentence):
    """If user's input is a greeting, return a greeting response"""
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [22]:
#Preprocessing Lemmatization
lemmer = WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [23]:
def response(url,query):
  

  headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}
  reg_url = url[0]
  req = urllib.request.Request(url=reg_url, headers=headers)
  
    
  out1=""
  html = urllib.request.urlopen(req).read()
  soup = BeautifulSoup(html, 'html.parser')
  paragraphs=soup.find_all('p')
  for para in paragraphs:
    data=para.get_text()
    data=re.sub(r'(\(adsbygo.*)',"",data)
    data=re.sub(r'[\n\t]',"",data)
    out1 = out1+data
  with open("data_set1.txt",'a',encoding='utf8') as file1:
    file1.write(out1)
    
  reg_url = url[1]
  req = urllib.request.Request(url=reg_url, headers=headers)  

  out2 = ""
  html = urllib.request.urlopen(req).read()
  soup = BeautifulSoup(html, 'html.parser')
  paragraphs=soup.find_all('p')
  for para in paragraphs:
    data=para.get_text()
    data=re.sub(r'(\(adsbygo.*)',"",data)
    data=re.sub(r'[\n\t]',"",data)
    out2 = out2+data
  with open("data_set2.txt",'a',encoding='utf8') as file1:
    file1.write(out2)
  
  reg_url = url[2]
  req = urllib.request.Request(url=reg_url, headers=headers)

  out3 = ""
  html = urllib.request.urlopen(req).read()
  soup = BeautifulSoup(html, 'html.parser')
  paragraphs=soup.find_all('p')
  for para in paragraphs:
    data=para.get_text()
    data=re.sub(r'(\(adsbygo.*)',"",data)
    data=re.sub(r'[\n\t]',"",data)
    out3 = out3+data
  with open("data_set3.txt",'a',encoding='utf8') as file1:
    file1.write(out3)

  return tfidf(query)

In [24]:
def tfidf(query):

  robo_response=''

  a1=open('data_set1.txt','r',errors = 'ignore')
  val1=a1.read()
  val1=val1.lower()# converts to lowercase
  sent_tokens_val1 = nltk.sent_tokenize(val1)# converts to list of sentences 
  word_tokens_val1 = nltk.word_tokenize(val1)

  a2=open('data_set2.txt','r',errors = 'ignore')
  val2=a2.read()
  val2=val2.lower()# converts to lowercase
  sent_tokens_val2 = nltk.sent_tokenize(val2)# converts to list of sentences 
  word_tokens_val2 = nltk.word_tokenize(val2)

  a3=open('data_set3.txt','r',errors = 'ignore')
  val3=a3.read()
  val3=val3.lower()# converts to lowercase
  sent_tokens_val3 = nltk.sent_tokenize(val3)# converts to list of sentences 
  word_tokens_val3 = nltk.word_tokenize(val3)

  sent_tokens_val1.append(query)
  sent_tokens_val2.append(query)
  sent_tokens_val3.append(query)

  TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')

  req_tfidf = []

  tfidf1 = TfidfVec.fit_transform(sent_tokens_val1)
  vals1 = cosine_similarity(tfidf1[-1], tfidf1)
  idx1=vals1.argsort()[0][-2]
  flat1 = vals1.flatten()
  flat1.sort()
  req_tfidf.append(flat1[-2])

  tfidf2 = TfidfVec.fit_transform(sent_tokens_val2)
  vals2 = cosine_similarity(tfidf2[-1], tfidf2)
  idx2=vals2.argsort()[0][-2]
  flat2 = vals2.flatten()
  flat2.sort()
  req_tfidf.append(flat2[-2])

  tfidf3 = TfidfVec.fit_transform(sent_tokens_val3)
  vals3 = cosine_similarity(tfidf3[-1], tfidf3)
  idx3=vals3.argsort()[0][-2]
  flat3 = vals3.flatten()
  flat3.sort()
  req_tfidf.append(flat3[-2])

  max_req_tfidf = max(req_tfidf)
  index = req_tfidf.index(max(req_tfidf))

  if index == 0:
    sent_tokens = sent_tokens_val1
    idx = idx1
  elif index == 1:
    sent_tokens = sent_tokens_val2
    idx = idx2
  elif index == 2:
    sent_tokens = sent_tokens_val3
    idx = idx3
  
  if(max_req_tfidf==0):
    robo_response=robo_response+"I am sorry! I don't understand you"
    return robo_response
  else:
    robo_response = robo_response+sent_tokens[idx]
    return robo_response

In [25]:
def google(query):
  url = []
  for j in search(query, tld="com", num=3, stop=3, pause=5):
    url.append(j)
  return response(url,query)

In [ ]:
flag=True
print("BRO: My name is BRO. I will answer your queries about Computers and AI. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye' and user_response != ''):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("BRO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("BRO: "+greeting(user_response))
            
            else:
                print("BRO: ",end="")
                print(google(user_response))
                os.remove("data_set1.txt")
                os.remove("data_set2.txt")
                os.remove("data_set3.txt")          
      
    else:
        flag=False
        print("BRO: Bye! take care..")

BRO: My name is BRO. I will answer your queries about Computers and AI. If you want to exit, type Bye!
what is macchine learning
BRO: so the purpose of reinforcement learning is to learn the best plan.there are many reinforcement learning algorithms are present in machine learning, which applied for different reinforcement learning applications.
what is machine learning
BRO: and now machine learning is present in so many segments of technology, that we donâ€™t even realise it while using it.machine learning mainly divided into three categories, which are as follows-supervised learning is the first type of machine learning, in which labelled data used to train the algorithms.
who is mahatma gandhi
BRO: these experiments took flak from most of the indian politicians and the people of india.favourite thingsfavourite personsgautama buddha, harishchandra, and his mother putlibai favourite authorleo tolstoy girls, affairs and moremarital statusmarriedaffairs/girlfriendsnot knownwifekasturba 